In [5]:
# -*- coding: UTF-8 -*- 
import onnx
from onnx import helper, checker
from onnx import TensorProto
import re
import argparse
import json,copy
import numpy as np
from data_json import *
from onnx_split import *

In [52]:
origin_model = '/home/sky/Downloads/models/densenet/densenet-7.onnx'
#origin_model = '/home/sky/Downloads/models/alexnet/bvlcalexnet-9.onnx'
platform_file = './4platform.txt'
nodes_number = 2


In [53]:
model = onnx.load(origin_model)
model = onnx.shape_inference.infer_shapes(model)
graph = model.graph


In [50]:
for input_i in range(len(graph.node[4].input)):
    graph.node[i].input[input_i] = graph.node[i].input[input_i].replace('/','_')

In [55]:
print (graph.input[0])

name: "conv1_w_0"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 64
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 7
      }
      dim {
        dim_value: 7
      }
    }
  }
}



In [57]:
for init_i in range(len(graph.initializer)):
        graph.initializer[init_i].name = graph.initializer[init_i].name.replace('/','_')
for i in range(len(graph.input)):
    graph.input[i].name = graph.input[i].name.replace('/','_')
for i in range(len(graph.output)):
    graph.output[i].name = graph.output[i].name.replace('/','_')
        
        
for i in range(len(graph.node)):
    graph.node[i].name = str(graph.node[i].output[0]).replace('/','_')
    for input_i in range(len(graph.node[i].input)):
        graph.node[i].input[input_i] = (graph.node[i].input[input_i]).replace('/','_')
    for output_i in range(len(graph.node[i].output)):
        graph.node[i].output[output_i] = graph.node[i].output[output_i].replace('/','_')
print("Check input model Errors: ", onnx.checker.check_model(model))
model_name = 'new'+origin_model.split('/')[-1]
print (model_name, " saved")
onnx.save(model, model_name)   

Check input model Errors:  None
newdensenet-7.onnx  saved


In [58]:
graph = load_onnx(model_name)
input_map = generate_node_dict(graph.input)  
output_map = generate_node_dict(graph.output)
initializer_map = generate_node_dict(graph.initializer)
value_map = generate_node_dict(graph.value_info)       
node_map = generate_node_dict(graph.node)
with open(platform_file, 'r') as f:         
        platform = f.readlines()
        #print (platform)
        for i in range(len(platform)):
            temp = platform[i].split(':')[0]
            platform[i] = temp.replace('\n','')
#print (platform)
platform_num = len(platform)

platform_dict={} 
######platform 对应id
for i in range(platform_num):
    platform_dict[platform[i]] = i
    
print ("platform dict: ", platform_dict)
node_len = len(list(node_map))
mapping_dict ={}


Check input model Errors:  None
platform dict:  {'nx01': 0, 'nx02': 1, 'nx03': 2, 'nx04': 3}


In [59]:
node_list = []
start_index = 0
end_index = 0
print ()
for i in range(nodes_number):  
    
    start_index = end_index
    end_index = min((int)(node_len/nodes_number+1)*(i+1) , node_len)
    #print (start_index, end_index)
    for j in range(start_index, end_index):
        node_list.append(graph.node[j].name)
    new_node_list = copy.copy(node_list)
    mapping_dict['nx0'+str(i+1)]=new_node_list
    node_list = []
print (mapping_dict)
save_json(mapping_dict,"./"+str(nodes_number)+"mapping.json")


{'nx01': ['conv1_1', 'conv1_bn_1', 'OC2_DUMMY_0', 'conv1_bn_internal_1', 'OC2_DUMMY_1', 'conv1_bn_2', 'conv1_bn_3', 'pool1_1', 'conv2_1_x1_bn_1', 'OC2_DUMMY_2', 'conv2_1_x1_bn_internal_1', 'OC2_DUMMY_3', 'conv2_1_x1_bn_2', 'conv2_1_x1_bn_3', 'conv2_1_x1_1', 'conv2_1_x2_bn_1', 'OC2_DUMMY_4', 'conv2_1_x2_bn_internal_1', 'OC2_DUMMY_5', 'conv2_1_x2_bn_2', 'conv2_1_x2_bn_3', 'conv2_1_x2_1', 'concat_2_1_1', 'conv2_2_x1_bn_1', 'OC2_DUMMY_6', 'conv2_2_x1_bn_internal_1', 'OC2_DUMMY_7', 'conv2_2_x1_bn_2', 'conv2_2_x1_bn_3', 'conv2_2_x1_1', 'conv2_2_x2_bn_1', 'OC2_DUMMY_8', 'conv2_2_x2_bn_internal_1', 'OC2_DUMMY_9', 'conv2_2_x2_bn_2', 'conv2_2_x2_bn_3', 'conv2_2_x2_1', 'concat_2_2_1', 'conv2_3_x1_bn_1', 'OC2_DUMMY_10', 'conv2_3_x1_bn_internal_1', 'OC2_DUMMY_11', 'conv2_3_x1_bn_2', 'conv2_3_x1_bn_3', 'conv2_3_x1_1', 'conv2_3_x2_bn_1', 'OC2_DUMMY_12', 'conv2_3_x2_bn_internal_1', 'OC2_DUMMY_13', 'conv2_3_x2_bn_2', 'conv2_3_x2_bn_3', 'conv2_3_x2_1', 'concat_2_3_1', 'conv2_4_x1_bn_1', 'OC2_DUMMY_14',